<a href="https://colab.research.google.com/github/TinizaraRodriguez/_easyOCR_paddleOCR/blob/main/easyOCR%26PaddleOCR_mejoraCalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr
!pip install paddlepaddle paddleocr
!pip install opencv-python
!pip install paddlepaddle-gpu
!pip install paddleocr
!pip install imutils

In [ ]:
import os
import easyocr
import cv2
from matplotlib import pyplot as plt
import numpy as np
import re
import regex
from PIL import Image
from google.colab.patches import cv2_imshow

#EasyOCR

Preparacion de EasyOCR cuando solo tenemos una imagen

In [ ]:
IMAGE_PATH = '/content/pantalan2.jpeg'
reader = easyocr.Reader(['es'])
result = reader.readtext(IMAGE_PATH)
result

In [ ]:
img = cv2.imread(IMAGE_PATH)
spacer = 100
font = cv2.FONT_HERSHEY_SIMPLEX
#matriculas = r'^[0-9]{2}-[A-Z]{2}-[0-9]-[0-9]{1,3}-[0-9]{2}$'

for detection in result:
  #if re.match(matriculas, detection[1]):
    top_left = tuple(map(int, detection[0][0]))
    bottom_right = tuple(map(int, detection[0][2]))
    text = detection[1]
    #new_text = re.sub(r'9', "a", text, 1)
    img = cv2.rectangle(img,top_left,bottom_right,(0,255,0),3)
    img = cv2.putText(img,text,(10,spacer), font, 2,(255,0,0),2,cv2.LINE_AA)
    spacer+=30
plt.figure(figsize=(15,15))
plt.imshow(img)
plt.show()


Ejecucion de EasyOCR - leyendo desde una carpeta e iterando sobre los elementos

In [ ]:
reader = easyocr.Reader(['es'])

# Tamaño mínimo de las imágenes
min_desired_size = (200, 50)

# Carpeta de entrada que contiene las imágenes
input_folder = '/content/inputs'

# Carpeta donde se guardarán los resultados
output_folder = '/content/outputs'

# Verificar si la carpeta de resultados existe, si no, créala
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Listar los archivos en la carpeta de entrada
image_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith(('.jpeg', '.jpg', '.png'))]

# Contador para el nombre de las imágenes
image_count = 1

# Iterar sobre los archivos de la carpeta de entrada
for image_path in image_files:
    # Cargar la imagen
    img = cv2.imread(image_path)

    # Verificar si la carga se ha realizado correctamente
    if img is None:
        print(f"Error: No se pudo cargar la imagen {image_path}")
        continue

    # Detección del texto
    results = reader.readtext(img)

    # Iterar sobre los resultados
    for (bbox, text, prob) in results:
        # Obtener las coordenadas
        top_left, top_right, bottom_right, bottom_left = bbox

        # Convertir las coordenadas a enteros
        top_left = tuple(map(int, top_left))
        bottom_right = tuple(map(int, bottom_right))

        # Dibujar el recuadro
        cv2.rectangle(img, top_left, bottom_right, (0, 255, 0), 2)

        # Recortar la región
        text_region = img[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]

        if not text_region is None and not text_region.size == 0:
            # Verificar si el tamaño es superior o igual al mínimo
            if text_region.shape[0] < min_desired_size[1] or text_region.shape[1] < min_desired_size[0]:
                # Redimensionar la región al tamaño dado
                text_region = cv2.resize(text_region, min_desired_size)

            # Guardar los recortes en la carpeta de salida con el nombre indicado
            output_filename = os.path.join(output_folder, f'output-{image_count}.jpg')
            cv2.imwrite(output_filename, text_region)
            image_count += 1

    # Mostrar la imagen final con los recuadros
    cv2_imshow(img)

# Esperar a que el usuario cierre las ventanas
cv2.waitKey(0)
cv2.destroyAllWindows()


#Mejora de la calidad de la imagen

Aplicamos escala de grises y contraste a las imagenes y las guardamos en una carpeta nueva que se usara como input para el paddleocr

In [ ]:
# Ruta de la carpeta de imágenes de entrada
input_folder = '/content/outputs'

# Ruta de la carpeta de imágenes de salida (donde se guardarán las imágenes mejoradas)
output_folder = '/content/outputs_improved'

# Verifica si la carpeta de salida existe, y si no, créala
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Obtener la lista de archivos en la carpeta de entrada
image_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith(('.jpeg', '.jpg', '.png'))]

# Función para mejorar la calidad de una imagen
def improve_image_quality(image):
    # Convertir la imagen a escala de grises
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar la mejora de contraste
    improved_image = cv2.equalizeHist(gray_image)

    return improved_image

# Procesar cada imagen en la carpeta de entrada
for img_path in image_files:
    # Cargar la imagen
    image = cv2.imread(img_path)

    if image is not None:
        # Aplicar mejoras de calidad a la imagen
        improved_image = improve_image_quality(image)

        # Guardar la imagen mejorada en la carpeta de salida
        output_path = os.path.join(output_folder, os.path.basename(img_path))
        cv2.imwrite(output_path, improved_image)

        print(f'Imagen mejorada guardada en: {output_path}')
    else:
        print(f"Error: No se pudo cargar la imagen {img_path}")

# Cerrar cualquier ventana abierta de OpenCV
cv2.destroyAllWindows()


#PaddleOCR

Evitar dependencias y clonacion del repo

In [ ]:
!wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.0g-2ubuntu4_amd64.deb
!sudo dpkg -i libssl1.1_1.1.0g-2ubuntu4_amd64.deb

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR

Ejecucion de PaddleOCR pasandole como input la carpeta con los recortes de las imagenes previas deteccion con easyocr

In [ ]:
from paddleocr import PaddleOCR, draw_ocr
import os

# Configuración de PaddleOCR
ocr = PaddleOCR(lang='es')

# Ruta de la carpeta de imágenes
input_folder = '/content/outputs_improved'

# Obtener la lista de archivos en la carpeta de imágenes
image_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith(('.jpeg', '.jpg', '.png'))]

# Procesar cada imagen en la carpeta
for img_path in image_files:
    # Realizar OCR en la imagen
    result = ocr.ocr(img_path, cls=False)

    if result:
        for idx in range(len(result)):
            res = result[idx]
            if res is not None:
                for line in res:
                    print(line)
            else:
                print(f"Advertencia: No se detectó texto en la imagen {img_path}")

        # Dibujar los resultados en la imagen original
        if result[0]:
            image = Image.open(img_path).convert('RGB')
            boxes = [line[0] for line in result[0]]
            txts = [line[1][0] for line in result[0]]
            scores = [line[1][1] for line in result[0]]
            im_show = draw_ocr(image, boxes, txts, scores, font_path='/content/PaddleOCR/doc/fonts/simfang.ttf')
            im_show = Image.fromarray(im_show)

            # Mostrar la imagen
            plt.figure(figsize=(20, 20))
            plt.imshow(im_show)
            plt.show()

            # Obtener el nombre original del archivo sin la extensión
            file_name_without_extension = os.path.splitext(os.path.basename(img_path))[0]

            # Guardar la imagen final con el nombre original más "output" como prefijo
            output_dir = '/content/outputs_final'
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, f'output_{file_name_without_extension}.jpg')
            im_show.save(output_path)
            print(f'Imagen final guardada en: {output_path}')
        else:
            print(f"Advertencia: No se encontraron resultados válidos para la imagen {img_path}")
    else:
        print(f"Advertencia: No se encontraron resultados válidos para la imagen {img_path}")
